In [1]:
#libraries
import re
import pandas as pd
import numpy as np
from scipy.stats import chisquare
import pysal.model.spreg as spreg
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pysal.lib
import geopandas as gpd
import pandas as pd
from libpysal.weights import DistanceBand
from scipy.spatial import distance_matrix
import pandas as pd
from scipy.stats import pearsonr
from esda.moran import Moran
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
import spreg
from spreg import SURlagIV
from spreg import GM_Lag
import libpysal
from spreg import AKtest
from spreg import ML_Lag
from spreg import GM_Combo

In [2]:
# lets read the metadata
with pd.io.stata.StataReader(r"C:\Users\soere\OneDrive\Desktop\Matserarbeit\Python\Variablen\Classification\control.dta") as reader:
    meta_control = reader.variable_labels()
    
with pd.io.stata.StataReader(r"C:\Users\soere\OneDrive\Desktop\Matserarbeit\Python\Variablen\Independent Variables\_FINAL_c_steamenginges_1904.dta") as reader:
    meta_steam = reader.variable_labels()

In [3]:
meta_control

{'id_inequality': 'County ID',
 'rb_id': 'District ID',
 'rb_name': 'District name',
 'province_id': 'Province ID',
 's_industry': 'Share industry employment',
 's_prot': 'Share Protestants',
 's_german': 'Share German-speaking',
 'longitud': 'Longitude',
 'latitud': 'Latitude',
 'slope_km': 'Terrain slope',
 'soil_suitability': 'Soil suitability',
 'mean_temp': 'Average temperature',
 'mean_preci': 'Average precipitation',
 'dist_100km': 'Distance to the nearest river',
 'log_distance_carbon_c': 'Least distance to carbon field (in log)'}

In [4]:
# Dependent variable
dep_var_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\2.2 Saving the Dependent Variables"

# Independent variable
ind_var_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\1.2 Saving the Independent Variables"

# Steamengines and horsepower
power_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\0. Pretransformed Data"

ifo_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\0.3 ifo Data"

In [5]:
# Load Skill levels per Landkreis
skills_df = pd.read_csv(f"{dep_var_path}\Main_Dep.csv",
                        sep=",")

# Load control variables
control_df = pd.read_csv(f"{ind_var_path}\Independent Variables.csv",
                         sep=",")

# Load SteamEngine and Horsepower data
power_df = pd.read_stata(f"{power_path}\_FINAL_c_steamenginges_1904.dta")

In [6]:
# Merge the dataframes using an inner join on the "id_inequality" column
prep_df = pd.merge(control_df, skills_df, on="id_inequality", how="inner")
merged_df = pd.merge(prep_df, power_df, on="id_inequality", how="inner")

In [7]:
merged_df

,id_inequality,rb_id,rb_name_x,province_id,s_industry,s_prot,s_german,longitud,latitud,slope_km,...,unskilled,Industriearbeiter,Anteil_Industriearbeiter,per_high,per_medium,per_low,per_unskilled,steamengines_number,horsepower,rb_name_y
0,1.0,17.0,Koenigsberg,1.0,18.515817,93.379268,56.527919,21.261999,55.648117,0.039,...,14043.295736,5482.0,0.214862,0.007630,0.218746,0.223209,0.550415,236.0,4499.0,Koenigsberg
1,2.0,17.0,Koenigsberg,1.0,20.088563,96.752059,99.449711,20.249264,54.843463,0.041,...,10586.366160,4673.0,0.231245,0.009308,0.225511,0.241311,0.523870,374.0,3653.0,Koenigsberg
2,4.0,17.0,Koenigsberg,1.0,35.498406,91.659959,99.395196,20.503550,54.706534,0.041,...,22103.884188,40793.0,0.435376,0.019338,0.361893,0.382858,0.235911,485.0,8563.0,Koenigsberg
3,5.0,17.0,Koenigsberg,1.0,15.060419,97.782661,99.669951,20.646094,54.737281,0.039,...,11623.154277,3063.0,0.172408,0.005532,0.202791,0.137440,0.654236,392.0,5820.0,Koenigsberg
4,6.0,17.0,Koenigsberg,1.0,12.007056,98.668155,89.094425,21.267903,54.855057,0.038,...,13898.117010,2839.0,0.136688,0.003992,0.198231,0.128634,0.669144,166.0,1581.0,Koenigsberg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,698.0,1.0,Aachen,12.0,15.278715,1.613553,70.916132,6.167253,50.364714,0.143,...,9385.982854,3334.0,0.158664,0.011330,0.162573,0.379421,0.446675,89.0,1548.0,Aachen
571,699.0,31.0,Sigmaringen,14.0,19.492555,5.190840,99.167226,9.235305,48.006617,0.165,...,5951.621706,2566.0,0.217053,0.007619,0.251187,0.237757,0.503436,57.0,488.0,Sigmaringen
572,700.0,31.0,Sigmaringen,14.0,23.974037,2.518404,99.882758,9.198332,48.235600,0.186,...,2289.005948,1144.0,0.256330,0.003951,0.228216,0.254948,0.512885,16.0,81.0,Sigmaringen
573,701.0,31.0,Sigmaringen,14.0,30.325834,6.910838,99.726559,9.001089,48.317343,0.182,...,6021.065087,3900.0,0.314719,0.004489,0.267196,0.242431,0.485883,76.0,876.0,Sigmaringen


In [8]:
# Drop Erblanden
#district_Erblanden = [31]
#merged_df = merged_df[~merged_df["rb_id"].isin(district_Erblanden)]

In [9]:
merged_df

,id_inequality,rb_id,rb_name_x,province_id,s_industry,s_prot,s_german,longitud,latitud,slope_km,...,unskilled,Industriearbeiter,Anteil_Industriearbeiter,per_high,per_medium,per_low,per_unskilled,steamengines_number,horsepower,rb_name_y
0,1.0,17.0,Koenigsberg,1.0,18.515817,93.379268,56.527919,21.261999,55.648117,0.039,...,14043.295736,5482.0,0.214862,0.007630,0.218746,0.223209,0.550415,236.0,4499.0,Koenigsberg
1,2.0,17.0,Koenigsberg,1.0,20.088563,96.752059,99.449711,20.249264,54.843463,0.041,...,10586.366160,4673.0,0.231245,0.009308,0.225511,0.241311,0.523870,374.0,3653.0,Koenigsberg
2,4.0,17.0,Koenigsberg,1.0,35.498406,91.659959,99.395196,20.503550,54.706534,0.041,...,22103.884188,40793.0,0.435376,0.019338,0.361893,0.382858,0.235911,485.0,8563.0,Koenigsberg
3,5.0,17.0,Koenigsberg,1.0,15.060419,97.782661,99.669951,20.646094,54.737281,0.039,...,11623.154277,3063.0,0.172408,0.005532,0.202791,0.137440,0.654236,392.0,5820.0,Koenigsberg
4,6.0,17.0,Koenigsberg,1.0,12.007056,98.668155,89.094425,21.267903,54.855057,0.038,...,13898.117010,2839.0,0.136688,0.003992,0.198231,0.128634,0.669144,166.0,1581.0,Koenigsberg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,698.0,1.0,Aachen,12.0,15.278715,1.613553,70.916132,6.167253,50.364714,0.143,...,9385.982854,3334.0,0.158664,0.011330,0.162573,0.379421,0.446675,89.0,1548.0,Aachen
571,699.0,31.0,Sigmaringen,14.0,19.492555,5.190840,99.167226,9.235305,48.006617,0.165,...,5951.621706,2566.0,0.217053,0.007619,0.251187,0.237757,0.503436,57.0,488.0,Sigmaringen
572,700.0,31.0,Sigmaringen,14.0,23.974037,2.518404,99.882758,9.198332,48.235600,0.186,...,2289.005948,1144.0,0.256330,0.003951,0.228216,0.254948,0.512885,16.0,81.0,Sigmaringen
573,701.0,31.0,Sigmaringen,14.0,30.325834,6.910838,99.726559,9.001089,48.317343,0.182,...,6021.065087,3900.0,0.314719,0.004489,0.267196,0.242431,0.485883,76.0,876.0,Sigmaringen


# Merge with ifo Data

#### Preparation

In [10]:
# Load the data from the selfmade dictionary
ifo_dict_own = pd.read_excel(f"{ifo_path}\ifo Dictionary Identification.xlsx", sheet_name="Tabelle1")

In [11]:
# Merge the county_names_df dataframe with the ifo_data to add the kreiskey1901 column
ifo_df = pd.merge(merged_df, ifo_dict_own, left_on="id_inequality", right_on="Match", how="left")
ifo_df

,id_inequality,rb_id,rb_name_x,province_id,s_industry,s_prot,s_german,longitud,latitud,slope_km,...,Anteil_Industriearbeiter,per_high,per_medium,per_low,per_unskilled,steamengines_number,horsepower,rb_name_y,kreiskey1901,Match
0,1.0,17.0,Koenigsberg,1.0,18.515817,93.379268,56.527919,21.261999,55.648117,0.039,...,0.214862,0.007630,0.218746,0.223209,0.550415,236.0,4499.0,Koenigsberg,1.0,1.0
1,2.0,17.0,Koenigsberg,1.0,20.088563,96.752059,99.449711,20.249264,54.843463,0.041,...,0.231245,0.009308,0.225511,0.241311,0.523870,374.0,3653.0,Koenigsberg,2.0,2.0
2,4.0,17.0,Koenigsberg,1.0,35.498406,91.659959,99.395196,20.503550,54.706534,0.041,...,0.435376,0.019338,0.361893,0.382858,0.235911,485.0,8563.0,Koenigsberg,3.0,4.0
3,5.0,17.0,Koenigsberg,1.0,15.060419,97.782661,99.669951,20.646094,54.737281,0.039,...,0.172408,0.005532,0.202791,0.137440,0.654236,392.0,5820.0,Koenigsberg,4.0,5.0
4,6.0,17.0,Koenigsberg,1.0,12.007056,98.668155,89.094425,21.267903,54.855057,0.038,...,0.136688,0.003992,0.198231,0.128634,0.669144,166.0,1581.0,Koenigsberg,5.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,698.0,1.0,Aachen,12.0,15.278715,1.613553,70.916132,6.167253,50.364714,0.143,...,0.158664,0.011330,0.162573,0.379421,0.446675,89.0,1548.0,Aachen,569.0,698.0
571,699.0,31.0,Sigmaringen,14.0,19.492555,5.190840,99.167226,9.235305,48.006617,0.165,...,0.217053,0.007619,0.251187,0.237757,0.503436,57.0,488.0,Sigmaringen,570.0,699.0
572,700.0,31.0,Sigmaringen,14.0,23.974037,2.518404,99.882758,9.198332,48.235600,0.186,...,0.256330,0.003951,0.228216,0.254948,0.512885,16.0,81.0,Sigmaringen,571.0,700.0
573,701.0,31.0,Sigmaringen,14.0,30.325834,6.910838,99.726559,9.001089,48.317343,0.182,...,0.314719,0.004489,0.267196,0.242431,0.485883,76.0,876.0,Sigmaringen,572.0,701.0


In [12]:
# Load the dictionary provided by the ifo
ifo_dict = pd.read_csv(f"{ifo_path}\\ipehd_merge_county.csv")

# Give ifo_df the identifier from the ifo_dict
# using kreiskey1901 as the identifier
ifo_df = pd.merge(ifo_df, 
                  ifo_dict[["kreiskey1901", "kreiskey1800"]], 
                  on="kreiskey1901", 
                  how="left")

# Drop rows where kreiskey1800 is NaN
ifo_df.dropna(subset=["kreiskey1800"], inplace=True)

In [13]:
# Create dictionary to match Regierungsbezirk and county in 1821
county_province_dict = ifo_df.set_index("kreiskey1800")["province_id"].to_dict()

ifo_df.columns

Index(['id_inequality', 'rb_id', 'rb_name_x', 'province_id', 's_industry',
       's_prot', 's_german', 'longitud', 'latitud', 'slope_km',
       'soil_suitability', 'mean_temp', 'mean_preci', 'dist_100km',
       'log_distance_carbon_c', 'NAME', 'county_name', 'uni_dummy',
       'min_distance_uni', 'prov_cap_dummy', 'BER_dummy', 'density',
       'd_Hauptberuf', 'high', 'medium', 'low', 'unskilled',
       'Industriearbeiter', 'Anteil_Industriearbeiter', 'per_high',
       'per_medium', 'per_low', 'per_unskilled', 'steamengines_number',
       'horsepower', 'rb_name_y', 'kreiskey1901', 'Match', 'kreiskey1800'],
      dtype='object')

In [14]:
# Summing up 'misc1821_area' for duplicate 'kreiskey1800' values and creating a single row for each
merged_df = ifo_df.groupby("kreiskey1800")[["log_distance_carbon_c",
                                                     "s_prot",
                                                     "s_german",
                                                     "longitud",
                                                     "latitud",
                                                     "slope_km",
                                                     "soil_suitability",
                                                     "mean_temp",
                                                     "mean_preci",
                                                     "dist_100km",
                                                     "min_distance_uni"]].mean().reset_index()

skills_sum = ifo_df.groupby("kreiskey1800")[["unskilled",
                                            "low",
                                            "medium",
                                            "high",
                                            "d_Hauptberuf",
                                            "steamengines_number",
                                            "horsepower"]].sum().reset_index()

BER_dummy = ifo_df.groupby("kreiskey1800")[["BER_dummy"]].max().reset_index()

merged_df = pd.merge(merged_df, 
                  skills_sum, 
                  on="kreiskey1800", 
                  how="left")

merged_df = pd.merge(merged_df, 
                  BER_dummy, 
                  on="kreiskey1800", 
                  how="left")

# Check the first few rows of the newly created dataframe
merged_df

,kreiskey1800,log_distance_carbon_c,s_prot,s_german,longitud,latitud,slope_km,soil_suitability,mean_temp,mean_preci,dist_100km,min_distance_uni,unskilled,low,medium,high,d_Hauptberuf,steamengines_number,horsepower,BER_dummy
0,1.0,5.358620,95.806330,99.799868,20.548731,54.397921,0.050000,2.102802,7.367000,59.828000,0.373000,0.442739,34883.768083,6767.603880,12209.372807,285.255230,54146.0,679.0,4861.0,0
1,2.0,5.567290,10.590991,79.509127,20.643597,53.937462,0.063000,1.413690,7.218000,52.488000,0.379333,0.808650,43982.647569,14941.215059,17452.082295,599.055076,76975.0,403.0,3315.0,0
2,3.0,5.621669,92.770560,85.218453,19.863912,53.859214,0.059667,1.819006,7.364667,55.057667,0.095667,1.089085,37435.262831,10471.527135,14675.508755,407.701280,62990.0,878.0,5976.0,0
3,4.0,5.621901,84.495686,27.943097,20.734716,53.462761,0.067000,2.251050,7.208500,46.101000,0.447500,1.308159,30516.418375,7691.764729,10824.645456,288.171440,49321.0,390.0,3151.0,0
4,5.0,5.421339,94.876874,99.482576,21.357758,54.253761,0.052000,1.875000,7.375500,49.400500,0.191500,0.974039,19429.061925,5474.878785,7029.235783,277.823507,32211.0,414.0,3419.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,277.0,1.997243,3.062395,98.907334,6.090387,50.649088,0.109000,3.666667,8.756000,83.165000,0.667000,0.110817,4371.460958,4155.018795,2733.636857,105.883390,11366.0,220.0,4209.0,0
276,278.0,2.785375,4.405721,99.762551,6.289372,50.605630,0.138000,5.000000,7.850000,86.592000,0.555000,0.244403,4678.245341,2509.927875,1729.825572,41.001212,8959.0,54.0,799.0,0
277,279.0,3.585311,4.088966,99.687772,6.560337,50.494628,0.135000,2.500000,7.907000,81.355000,0.472000,0.531539,10651.028190,5293.003586,4607.236824,142.731400,20694.0,253.0,5686.0,0
278,280.0,3.452736,1.613553,70.916132,6.167253,50.364714,0.143000,2.785714,7.611000,93.091000,0.724000,0.400585,9385.982854,7972.780302,3416.156704,238.080140,21013.0,89.0,1548.0,0


#### Area and population data to create density

In [15]:
# Area data
ifo_area_df = pd.read_csv(f"{ifo_path}\\ipehd_1821_misc_area.csv", encoding='ISO-8859-1')
# Summing up areas for duplicates
ifo_area_df = ifo_area_df.groupby('kreiskey1800')['misc1821_area'].sum().reset_index()

# Check the first few rows of the newly created dataframe
ifo_area_df

,kreiskey1800,misc1821_area
0,1,1143405.0
1,2,1265474.0
2,3,1189582.0
3,4,1229218.0
4,5,634855.0
...,...,...
275,277,85005.0
276,278,55961.0
277,279,119590.0
278,280,276751.0


In [16]:
merged_df = pd.merge(merged_df, ifo_area_df[["kreiskey1800", "misc1821_area"]], 
                                         on="kreiskey1800", 
                                         how="left")

In [17]:
# Population data on underage in 1816 (females only until 14)
ifo_pop_1816_df = pd.read_csv(f"{ifo_path}\ipehd_1816_pop_demo.csv", encoding="ISO-8859-1")
ifo_pop_1816_df["underage_m"] = ifo_pop_1816_df["pop1816_m_un7"] + ifo_pop_1816_df["pop1816_m_7to14"] + ifo_pop_1816_df["pop1816_m_15to18"]
ifo_pop_1816_df["underage_f"] = ifo_pop_1816_df["pop1816_f_un7"] + ifo_pop_1816_df["pop1816_f_7to14"]
ifo_pop_1816_df["underage"] = ifo_pop_1816_df["underage_f"] + ifo_pop_1816_df["underage_m"]

ifo_pop_1816_df = ifo_pop_1816_df.groupby("kreiskey1800")[["underage"]].sum().reset_index()

# Check the first few rows of the newly created dataframe
ifo_pop_1816_df

,kreiskey1800,underage
0,1,25316.0
1,2,26417.0
2,3,27430.0
3,4,22179.0
4,5,22867.0
...,...,...
275,277,17441.0
276,278,6898.0
277,279,6860.0
278,280,11907.0


In [18]:
merged_df = pd.merge(merged_df, ifo_pop_1816_df, on="kreiskey1800", how="left")

In [19]:
# Population data
ifo_pop_df = pd.read_csv(f"{ifo_path}\\ipehd_1821_pop_demo.csv", encoding='ISO-8859-1')
# Summing up 'misc1821_area' for duplicate 'kreiskey1800' values and creating a single row for each
ifo_pop_df = ifo_pop_df.groupby('kreiskey1800')['pop1821_tot'].sum().reset_index()

# Check the first few rows of the newly created dataframe
ifo_pop_df


,kreiskey1800,pop1821_tot
0,1,85831.0
1,2,88389.0
2,3,90056.0
3,4,63720.0
4,5,52195.0
...,...,...
275,277,45877.0
276,278,17143.0
277,279,17439.0
278,280,29726.0


In [20]:
merged_df = pd.merge(merged_df, 
                     ifo_pop_df[["kreiskey1800", "pop1821_tot"]], 
                     on="kreiskey1800", 
                     how="left")

#merged_county_names_with_area.dropna(subset=["misc1821_area"], inplace=True)
merged_df["density"] = merged_df["pop1821_tot"] / merged_df["misc1821_area"]
merged_df

,kreiskey1800,log_distance_carbon_c,s_prot,s_german,longitud,latitud,slope_km,soil_suitability,mean_temp,mean_preci,...,medium,high,d_Hauptberuf,steamengines_number,horsepower,BER_dummy,misc1821_area,underage,pop1821_tot,density
0,1.0,5.358620,95.806330,99.799868,20.548731,54.397921,0.050000,2.102802,7.367000,59.828000,...,12209.372807,285.255230,54146.0,679.0,4861.0,0,1143405.0,25316.0,85831.0,0.075066
1,2.0,5.567290,10.590991,79.509127,20.643597,53.937462,0.063000,1.413690,7.218000,52.488000,...,17452.082295,599.055076,76975.0,403.0,3315.0,0,1265474.0,26417.0,88389.0,0.069847
2,3.0,5.621669,92.770560,85.218453,19.863912,53.859214,0.059667,1.819006,7.364667,55.057667,...,14675.508755,407.701280,62990.0,878.0,5976.0,0,1189582.0,27430.0,90056.0,0.075704
3,4.0,5.621901,84.495686,27.943097,20.734716,53.462761,0.067000,2.251050,7.208500,46.101000,...,10824.645456,288.171440,49321.0,390.0,3151.0,0,1229218.0,22179.0,63720.0,0.051838
4,5.0,5.421339,94.876874,99.482576,21.357758,54.253761,0.052000,1.875000,7.375500,49.400500,...,7029.235783,277.823507,32211.0,414.0,3419.0,0,634855.0,22867.0,52195.0,0.082216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,277.0,1.997243,3.062395,98.907334,6.090387,50.649088,0.109000,3.666667,8.756000,83.165000,...,2733.636857,105.883390,11366.0,220.0,4209.0,0,85005.0,17441.0,45877.0,0.539698
276,278.0,2.785375,4.405721,99.762551,6.289372,50.605630,0.138000,5.000000,7.850000,86.592000,...,1729.825572,41.001212,8959.0,54.0,799.0,0,55961.0,6898.0,17143.0,0.306338
277,279.0,3.585311,4.088966,99.687772,6.560337,50.494628,0.135000,2.500000,7.907000,81.355000,...,4607.236824,142.731400,20694.0,253.0,5686.0,0,119590.0,6860.0,17439.0,0.145823
278,280.0,3.452736,1.613553,70.916132,6.167253,50.364714,0.143000,2.785714,7.611000,93.091000,...,3416.156704,238.080140,21013.0,89.0,1548.0,0,276751.0,11907.0,29726.0,0.107411


#### Education data from 1821

In [21]:
# Population data
ifo_students_df = pd.read_csv(f"{ifo_path}\\ipehd_1816_edu_stud.csv", encoding="ISO-8859-1")

# Middle and high school
student_types_h = ["edu1816_pub_mim_stud_m",
       "edu1816_pub_mif_stud_f", "edu1816_pri_mim_stud_m",
       "edu1816_pri_mif_stud_f", "edu1816_pub_gel_stud_m"]

ifo_students_df["high_stud"] = 0

for col in student_types_h:
    ifo_students_df["high_stud"] = ifo_students_df["high_stud"] + ifo_students_df[col]

# Primary schooling    
student_types_l = ["edu1816_pub_ele_stud_m",
       "edu1816_pub_ele_stud_f", "edu1816_pri_ele_stud_m",
       "edu1816_pri_ele_stud_f"]

ifo_students_df["low_stud"] = 0

for col in student_types_l:
    ifo_students_df["low_stud"] = ifo_students_df["low_stud"] + ifo_students_df[col]
    

ifo_students_df = ifo_students_df.groupby("kreiskey1800")[["low_stud","high_stud"]].sum().reset_index()

# Check the first few rows of the newly created dataframe
ifo_students_df

,kreiskey1800,low_stud,high_stud
0,1,7593.0,0.0
1,2,2410.0,0.0
2,3,9580.0,0.0
3,4,7249.0,0.0
4,5,6156.0,0.0
...,...,...,...
275,277,2485.0,728.0
276,278,865.0,99.0
277,279,995.0,0.0
278,280,3327.0,0.0


In [22]:
merged_df = pd.merge(merged_df, ifo_students_df, on="kreiskey1800", how="left")

In [23]:
# Calculate student share
merged_df["s_high_stud"] = merged_df["high_stud"]/merged_df["underage"]
merged_df["s_low_stud"] = merged_df["low_stud"]/merged_df["underage"]

In [24]:
merged_df

,kreiskey1800,log_distance_carbon_c,s_prot,s_german,longitud,latitud,slope_km,soil_suitability,mean_temp,mean_preci,...,horsepower,BER_dummy,misc1821_area,underage,pop1821_tot,density,low_stud,high_stud,s_high_stud,s_low_stud
0,1.0,5.358620,95.806330,99.799868,20.548731,54.397921,0.050000,2.102802,7.367000,59.828000,...,4861.0,0,1143405.0,25316.0,85831.0,0.075066,7593.0,0.0,0.000000,0.299929
1,2.0,5.567290,10.590991,79.509127,20.643597,53.937462,0.063000,1.413690,7.218000,52.488000,...,3315.0,0,1265474.0,26417.0,88389.0,0.069847,2410.0,0.0,0.000000,0.091229
2,3.0,5.621669,92.770560,85.218453,19.863912,53.859214,0.059667,1.819006,7.364667,55.057667,...,5976.0,0,1189582.0,27430.0,90056.0,0.075704,9580.0,0.0,0.000000,0.349253
3,4.0,5.621901,84.495686,27.943097,20.734716,53.462761,0.067000,2.251050,7.208500,46.101000,...,3151.0,0,1229218.0,22179.0,63720.0,0.051838,7249.0,0.0,0.000000,0.326841
4,5.0,5.421339,94.876874,99.482576,21.357758,54.253761,0.052000,1.875000,7.375500,49.400500,...,3419.0,0,634855.0,22867.0,52195.0,0.082216,6156.0,0.0,0.000000,0.269209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,277.0,1.997243,3.062395,98.907334,6.090387,50.649088,0.109000,3.666667,8.756000,83.165000,...,4209.0,0,85005.0,17441.0,45877.0,0.539698,2485.0,728.0,0.041741,0.142480
276,278.0,2.785375,4.405721,99.762551,6.289372,50.605630,0.138000,5.000000,7.850000,86.592000,...,799.0,0,55961.0,6898.0,17143.0,0.306338,865.0,99.0,0.014352,0.125399
277,279.0,3.585311,4.088966,99.687772,6.560337,50.494628,0.135000,2.500000,7.907000,81.355000,...,5686.0,0,119590.0,6860.0,17439.0,0.145823,995.0,0.0,0.000000,0.145044
278,280.0,3.452736,1.613553,70.916132,6.167253,50.364714,0.143000,2.785714,7.611000,93.091000,...,1548.0,0,276751.0,11907.0,29726.0,0.107411,3327.0,0.0,0.000000,0.279415


#### Public utility buildings

In [25]:
ifo_build_df = pd.read_csv(f"{ifo_path}\ipehd_1821_misc_buil.csv", encoding="ISO-8859-1")

In [26]:
ifo_build_df = ifo_build_df.groupby("kreiskey1800")["buil1821_pub"].sum().reset_index()

In [27]:
merged_df = pd.merge(merged_df, ifo_build_df, on="kreiskey1800", how="left")

In [28]:
merged_df["s_build"] = merged_df["buil1821_pub"]/merged_df["pop1821_tot"]
merged_df

,kreiskey1800,log_distance_carbon_c,s_prot,s_german,longitud,latitud,slope_km,soil_suitability,mean_temp,mean_preci,...,misc1821_area,underage,pop1821_tot,density,low_stud,high_stud,s_high_stud,s_low_stud,buil1821_pub,s_build
0,1.0,5.358620,95.806330,99.799868,20.548731,54.397921,0.050000,2.102802,7.367000,59.828000,...,1143405.0,25316.0,85831.0,0.075066,7593.0,0.0,0.000000,0.299929,284.0,0.003309
1,2.0,5.567290,10.590991,79.509127,20.643597,53.937462,0.063000,1.413690,7.218000,52.488000,...,1265474.0,26417.0,88389.0,0.069847,2410.0,0.0,0.000000,0.091229,158.0,0.001788
2,3.0,5.621669,92.770560,85.218453,19.863912,53.859214,0.059667,1.819006,7.364667,55.057667,...,1189582.0,27430.0,90056.0,0.075704,9580.0,0.0,0.000000,0.349253,325.0,0.003609
3,4.0,5.621901,84.495686,27.943097,20.734716,53.462761,0.067000,2.251050,7.208500,46.101000,...,1229218.0,22179.0,63720.0,0.051838,7249.0,0.0,0.000000,0.326841,348.0,0.005461
4,5.0,5.421339,94.876874,99.482576,21.357758,54.253761,0.052000,1.875000,7.375500,49.400500,...,634855.0,22867.0,52195.0,0.082216,6156.0,0.0,0.000000,0.269209,173.0,0.003314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,277.0,1.997243,3.062395,98.907334,6.090387,50.649088,0.109000,3.666667,8.756000,83.165000,...,85005.0,17441.0,45877.0,0.539698,2485.0,728.0,0.041741,0.142480,50.0,0.001090
276,278.0,2.785375,4.405721,99.762551,6.289372,50.605630,0.138000,5.000000,7.850000,86.592000,...,55961.0,6898.0,17143.0,0.306338,865.0,99.0,0.014352,0.125399,30.0,0.001750
277,279.0,3.585311,4.088966,99.687772,6.560337,50.494628,0.135000,2.500000,7.907000,81.355000,...,119590.0,6860.0,17439.0,0.145823,995.0,0.0,0.000000,0.145044,44.0,0.002523
278,280.0,3.452736,1.613553,70.916132,6.167253,50.364714,0.143000,2.785714,7.611000,93.091000,...,276751.0,11907.0,29726.0,0.107411,3327.0,0.0,0.000000,0.279415,101.0,0.003398


#### Skill shares

In [29]:
merged_df["per_unskilled"] = merged_df["unskilled"] / merged_df["d_Hauptberuf"]
merged_df["per_low"] = merged_df["low"] / merged_df["d_Hauptberuf"]
merged_df["per_medium"] = merged_df["medium"] / merged_df["d_Hauptberuf"]
merged_df["per_high"] = merged_df["high"] / merged_df["d_Hauptberuf"]

#### Landownership concentration

In [30]:
ifo_agri_df = pd.read_csv(f"{ifo_path}\ipehd_1816_agri_land.csv", encoding="ISO-8859-1")
ifo_agri_df

,id1816,kreiskey1800,county,rb,land1816_far_ext_ov300,land1816_far_ext_15to30,land1816_far_ext_un15
0,1,277,Achen,AAC,7.0,881.0,2831.0
1,2,33,Adelnau,POS,100.0,1807.0,2027.0
2,3,254,Adenau,KOB,0.0,798.0,2805.0
3,4,196,Ahaus,MUN,5.0,197.0,3313.0
4,5,255,Ahrweiler,KOB,2.0,572.0,4385.0
...,...,...,...,...,...,...,...
303,304,276,Aachen,AAC,NaN,NaN,NaN
304,305,228,Köln,KOL,NaN,NaN,NaN
305,306,193,Münster,MUN,NaN,NaN,NaN
306,307,59,Potsdam,POT,0.0,14.0,48.0


In [31]:
ifo_agri_df = ifo_agri_df.groupby("kreiskey1800")[["land1816_far_ext_ov300", "land1816_far_ext_15to30", "land1816_far_ext_un15"]].sum().reset_index()

In [32]:
list_agri = ["land1816_far_ext_ov300", "land1816_far_ext_15to30", "land1816_far_ext_un15"]

ifo_agri_df["total_agri"] = 0

for col in list_agri:
    ifo_agri_df["total_agri"] = ifo_agri_df["total_agri"]+ifo_agri_df[col]

ifo_agri_df["s_agri"] = ifo_agri_df["land1816_far_ext_ov300"] / ifo_agri_df["total_agri"]
ifo_agri_df["s_agri"] = ifo_agri_df["s_agri"].astype(float)
ifo_agri_df

,kreiskey1800,land1816_far_ext_ov300,land1816_far_ext_15to30,land1816_far_ext_un15,total_agri,s_agri
0,1,241.0,2946.0,921.0,4108.0,0.058666
1,2,67.0,4629.0,1049.0,5745.0,0.011662
2,3,195.0,3274.0,729.0,4198.0,0.046451
3,4,105.0,5404.0,450.0,5959.0,0.017620
4,5,106.0,2463.0,477.0,3046.0,0.034800
...,...,...,...,...,...,...
275,277,7.0,881.0,2831.0,3719.0,0.001882
276,278,5.0,535.0,1525.0,2065.0,0.002421
277,279,0.0,318.0,2294.0,2612.0,0.000000
278,280,3.0,1128.0,3374.0,4505.0,0.000666


In [33]:
merged_df = pd.merge(merged_df, ifo_agri_df, on="kreiskey1800", how="left")

#### Water Mills

In [34]:
ifo_mill_df = pd.read_csv(f"{ifo_path}\ipehd_1819_indu_fac.csv", encoding="ISO-8859-1")
ifo_mill_df

,id1819,kreiskey1800,county,rb,fac1819_brick,fac1819_lime,fac1819_glass,mill1819_water,mill1819_saw_one,mill1819_saw_more
0,1,277,Achen,AAC,5,10,2,26,0,0
1,2,33,Adelnau,POS,11,6,0,26,1,0
2,3,254,Adenau,KOB,0,1,0,71,0,0
3,4,196,Ahaus,MUN,11,15,0,20,0,0
4,5,255,Ahrweiler,KOB,0,0,0,51,0,0
...,...,...,...,...,...,...,...,...,...,...
339,340,276,Aachen,AAC,10,1,1,10,0,0
340,341,228,Köln,KOL,0,0,1,4,0,0
341,342,193,Münster,MUN,0,0,0,2,0,0
342,343,59,Potsdam,POT,0,2,0,2,0,0


In [35]:
ifo_mill_df = ifo_mill_df.groupby("kreiskey1800")[["mill1819_water"]].sum().reset_index()

In [36]:
merged_df = pd.merge(merged_df, ifo_mill_df, on="kreiskey1800", how="left")

In [37]:
merged_df["s_mill"] = merged_df["mill1819_water"]/merged_df["pop1821_tot"]

#### Check for NaNs

had to drop some observations

In [38]:
nan_columns = merged_df.isna().any()
nan_sum = merged_df.isna().sum()
print(nan_columns)
print(nan_sum)

kreiskey1800               False
log_distance_carbon_c      False
s_prot                     False
s_german                   False
longitud                   False
latitud                    False
slope_km                   False
soil_suitability           False
mean_temp                  False
mean_preci                 False
dist_100km                 False
min_distance_uni           False
unskilled                  False
low                        False
medium                     False
high                       False
d_Hauptberuf               False
steamengines_number        False
horsepower                 False
BER_dummy                  False
misc1821_area              False
underage                   False
pop1821_tot                False
density                    False
low_stud                   False
high_stud                  False
s_high_stud                 True
s_low_stud                  True
buil1821_pub               False
s_build                    False
per_unskil

In [39]:
merged_df = merged_df.dropna(subset=["s_high_stud"])
merged_df['s_high_stud'] = merged_df['s_high_stud'].replace([np.inf, -np.inf], 0)
len(merged_df)

270

In [40]:
merged_df = merged_df.dropna(subset=["s_low_stud"])
merged_df = merged_df.dropna(subset=["s_agri"])
#merged_df = merged_df[~merged_df["s_high_stud"].isin([np.inf, -np.inf])]

# Spatial Autocorrelation
First I check for spatial autocorrelation

In [41]:
gdf = gpd.GeoDataFrame(merged_df, geometry=gpd.points_from_xy(merged_df.longitud, merged_df.latitud))

In [42]:
# Compute the distance matrix
dist_matrix = distance_matrix(merged_df[["longitud", "latitud"]], merged_df[["longitud", "latitud"]])

# Compute the mean, median and 99th percentile distance
mean_distance = dist_matrix.mean()
median_distance = np.median(dist_matrix)  # Use numpy's median function as pd.median() operates on DataFrame/Series
percentile = np.percentile(dist_matrix, 99)

# Output
print(f"Mean Distance: {mean_distance}")
print(f"Median Distance: {median_distance}")
print(f"99th percentile: {percentile}")

Mean Distance: 5.4037625856801155
Median Distance: 4.8027921525902775
99th percentile: 14.526246265338697


In [43]:
# Creating spatial weights matrix (Euclidean distance)
w = DistanceBand.from_dataframe(gdf, threshold=median_distance, binary=False)

# Transform the matrix so all rows add up to 1
w.transform = "r"

C:\Users\soere\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\sparse\_data.py:117: RuntimeWarning: divide by zero encountered in reciprocal
  return self._with_data(data ** n)


In [44]:
merged_df.columns

Index(['kreiskey1800', 'log_distance_carbon_c', 's_prot', 's_german',
       'longitud', 'latitud', 'slope_km', 'soil_suitability', 'mean_temp',
       'mean_preci', 'dist_100km', 'min_distance_uni', 'unskilled', 'low',
       'medium', 'high', 'd_Hauptberuf', 'steamengines_number', 'horsepower',
       'BER_dummy', 'misc1821_area', 'underage', 'pop1821_tot', 'density',
       'low_stud', 'high_stud', 's_high_stud', 's_low_stud', 'buil1821_pub',
       's_build', 'per_unskilled', 'per_low', 'per_medium', 'per_high',
       'land1816_far_ext_ov300', 'land1816_far_ext_15to30',
       'land1816_far_ext_un15', 'total_agri', 's_agri', 'mill1819_water',
       's_mill'],
      dtype='object')

In [45]:
# Check variables for spatial autocorrelation
mi_carbon = Moran(gdf["log_distance_carbon_c"], w)
mi_steam = Moran((gdf["steamengines_number"]/gdf["d_Hauptberuf"]), w)
mi_hp = Moran((gdf["horsepower"]/gdf["d_Hauptberuf"]), w)
mi_unskilled = Moran(gdf["per_unskilled"], w)
mi_low = Moran(gdf["per_low"], w)
mi_medium = Moran(gdf["per_medium"], w)
mi_high = Moran(gdf["per_high"], w)

# Output Moran's I value and p-value
print(f"log_distance_carbon_c - Moran's I: {mi_carbon.I}, p-value: {mi_carbon.p_sim}")
print(f"steamengines_number - Moran's I: {mi_steam.I}, p-value: {mi_steam.p_sim}")
print(f"horsepower - Moran's I: {mi_hp.I}, p-value: {mi_hp.p_sim}")
print(f"Unskilled - Moran's I: {mi_hp.I}, p-value: {mi_hp.p_sim}")
print(f"Low - Moran's I: {mi_hp.I}, p-value: {mi_hp.p_sim}")
print(f"Medium - Moran's I: {mi_hp.I}, p-value: {mi_hp.p_sim}")
print(f"High - Moran's I: {mi_hp.I}, p-value: {mi_hp.p_sim}")

log_distance_carbon_c - Moran's I: 0.3677952564115726, p-value: 0.001
steamengines_number - Moran's I: 0.11376551564719149, p-value: 0.001
horsepower - Moran's I: 0.1677225992623932, p-value: 0.001
Unskilled - Moran's I: 0.1677225992623932, p-value: 0.001
Low - Moran's I: 0.1677225992623932, p-value: 0.001
Medium - Moran's I: 0.1677225992623932, p-value: 0.001
High - Moran's I: 0.1677225992623932, p-value: 0.001


# 2SLS
Lets run a good'ol reliable IV first

In [46]:
# Transform steamengines to steamengines per worker
merged_df["steamengines_number"] = (merged_df["steamengines_number"] / merged_df["d_Hauptberuf"])*1000
merged_df["horsepower"] = (merged_df["horsepower"] / merged_df["d_Hauptberuf"])*1000

# Log some variables and add 1 if they contain zeros

merged_df["steamengines_number"] = np.log(merged_df["steamengines_number"])
merged_df["horsepower"] = np.log(merged_df["horsepower"])

In [47]:
merged_df.columns

Index(['kreiskey1800', 'log_distance_carbon_c', 's_prot', 's_german',
       'longitud', 'latitud', 'slope_km', 'soil_suitability', 'mean_temp',
       'mean_preci', 'dist_100km', 'min_distance_uni', 'unskilled', 'low',
       'medium', 'high', 'd_Hauptberuf', 'steamengines_number', 'horsepower',
       'BER_dummy', 'misc1821_area', 'underage', 'pop1821_tot', 'density',
       'low_stud', 'high_stud', 's_high_stud', 's_low_stud', 'buil1821_pub',
       's_build', 'per_unskilled', 'per_low', 'per_medium', 'per_high',
       'land1816_far_ext_ov300', 'land1816_far_ext_15to30',
       'land1816_far_ext_un15', 'total_agri', 's_agri', 'mill1819_water',
       's_mill'],
      dtype='object')

## Try

In [48]:
merged_df["per_unskilled"] = merged_df["per_unskilled"]*100
merged_df["per_low"] = merged_df["per_low"]*100
merged_df["per_medium"] = merged_df["per_medium"]*100
merged_df["per_high"] = merged_df["per_high"]*100

In [49]:
merged_df["per_unskilled"] = np.log(merged_df["per_unskilled"])
merged_df["per_low"] = np.log(merged_df["per_low"])
merged_df["per_medium"] = np.log(merged_df["per_medium"])
merged_df["per_high"] = np.log(merged_df["per_high"])

In [50]:
merged_df

,kreiskey1800,log_distance_carbon_c,s_prot,s_german,longitud,latitud,slope_km,soil_suitability,mean_temp,mean_preci,...,per_low,per_medium,per_high,land1816_far_ext_ov300,land1816_far_ext_15to30,land1816_far_ext_un15,total_agri,s_agri,mill1819_water,s_mill
0,1.0,5.358620,95.806330,99.799868,20.548731,54.397921,0.050000,2.102802,7.367000,59.828000,...,2.525633,3.115690,-0.640885,241.0,2946.0,921.0,4108.0,0.058666,66,0.000769
1,2.0,5.567290,10.590991,79.509127,20.643597,53.937462,0.063000,1.413690,7.218000,52.488000,...,2.965813,3.121148,-0.250712,67.0,4629.0,1049.0,5745.0,0.011662,67,0.000758
2,3.0,5.621669,92.770560,85.218453,19.863912,53.859214,0.059667,1.819006,7.364667,55.057667,...,2.810854,3.148374,-0.435026,195.0,3274.0,729.0,4198.0,0.046451,77,0.000855
3,4.0,5.621901,84.495686,27.943097,20.734716,53.462761,0.067000,2.251050,7.208500,46.101000,...,2.746970,3.088646,-0.537379,105.0,5404.0,450.0,5959.0,0.017620,40,0.000628
4,5.0,5.421339,94.876874,99.482576,21.357758,54.253761,0.052000,1.875000,7.375500,49.400500,...,2.833032,3.082940,-0.147907,106.0,2463.0,477.0,3046.0,0.034800,30,0.000575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,277.0,1.997243,3.062395,98.907334,6.090387,50.649088,0.109000,3.666667,8.756000,83.165000,...,3.598861,3.180177,-0.070873,7.0,881.0,2831.0,3719.0,0.001882,26,0.000567
276,278.0,2.785375,4.405721,99.762551,6.289372,50.605630,0.138000,5.000000,7.850000,86.592000,...,3.332766,2.960532,-0.781642,5.0,535.0,1525.0,2065.0,0.002421,14,0.000817
277,279.0,3.585311,4.088966,99.687772,6.560337,50.494628,0.135000,2.500000,7.907000,81.355000,...,3.241712,3.102955,-0.371464,0.0,318.0,2294.0,2612.0,0.000000,15,0.000860
278,280.0,3.452736,1.613553,70.916132,6.167253,50.364714,0.143000,2.785714,7.611000,93.091000,...,3.636062,2.788545,0.124881,3.0,1128.0,3374.0,4505.0,0.000666,62,0.002086


# Create Dummy for FE

In [51]:
merged_df["province_id"] = merged_df["kreiskey1800"].map(county_province_dict)

dummies = pd.get_dummies(merged_df["province_id"], drop_first=True) * 1
merged_df = pd.concat([merged_df, dummies], axis=1)

# Convert column names to strings
merged_df.columns = [str(col) for col in merged_df.columns]
# Dictionary
number_words = {
    "1": "one", "2": "two", "3": "three", "4": "four", "5": "five",
    "6": "six", "7": "seven", "8": "eight", "9": "nine", "10": "ten",
    "11": "eleven", "12": "twelve", "13": "thirteen", "14": "fourteen"
}

# Transform the column names
merged_df.columns = [number_words[col.split(".")[0]] if col.split(".")[0] in number_words else col for col in merged_df.columns]

merged_df

,kreiskey1800,log_distance_carbon_c,s_prot,s_german,longitud,latitud,slope_km,soil_suitability,mean_temp,mean_preci,...,two,three,four,five,six,seven,ten,twelve,thirteen,fourteen
0,1.0,5.358620,95.806330,99.799868,20.548731,54.397921,0.050000,2.102802,7.367000,59.828000,...,0,0,0,0,0,0,0,0,0,0
1,2.0,5.567290,10.590991,79.509127,20.643597,53.937462,0.063000,1.413690,7.218000,52.488000,...,0,0,0,0,0,0,0,0,0,0
2,3.0,5.621669,92.770560,85.218453,19.863912,53.859214,0.059667,1.819006,7.364667,55.057667,...,0,0,0,0,0,0,0,0,0,0
3,4.0,5.621901,84.495686,27.943097,20.734716,53.462761,0.067000,2.251050,7.208500,46.101000,...,0,0,0,0,0,0,0,0,0,0
4,5.0,5.421339,94.876874,99.482576,21.357758,54.253761,0.052000,1.875000,7.375500,49.400500,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,277.0,1.997243,3.062395,98.907334,6.090387,50.649088,0.109000,3.666667,8.756000,83.165000,...,0,0,0,0,0,0,0,1,0,0
276,278.0,2.785375,4.405721,99.762551,6.289372,50.605630,0.138000,5.000000,7.850000,86.592000,...,0,0,0,0,0,0,0,1,0,0
277,279.0,3.585311,4.088966,99.687772,6.560337,50.494628,0.135000,2.500000,7.907000,81.355000,...,0,0,0,0,0,0,0,1,0,0
278,280.0,3.452736,1.613553,70.916132,6.167253,50.364714,0.143000,2.785714,7.611000,93.091000,...,0,0,0,0,0,0,0,1,0,0


## 2SLS function
I build a function that runs multiple 2SLS in a row, each time adding another variable from a list of control variables

In [52]:
def twoSLS(df, IV, first_stage_dependent, controls, second_stage_independent):
    """
    Performs a series of Two-Stage Least Squares (2SLS) regressions on a given DataFrame.
    In each iteration, an additional control variable from the provided list is included.

    Parameters:
    df (DataFrame): The DataFrame containing the data for regression.
    IV (str): The name of the instrumental variable column in the DataFrame.
    first_stage_dependent (str): The name of the dependent variable for the first stage regression.
    controls (list of str): A list of control variable names. Each iteration includes an additional control variable.
    second_stage_independent (str): The name of the main independent variable for the second stage regression.

    Returns:
    tuple of DataFrames: The first DataFrame contains the results (coefficients, significance, 
                         Adj. R-squared, F-statistic) of each first stage regression. 
                         The second DataFrame contains the results 
                         of each second stage regression. Significance is denoted by stars: 
                         *** for p < 0.01, ** for p < 0.05, and * for p < 0.1.
                         The third DataFrame contains the residuals.
    """

    first_stage_results = []
    second_stage_results = []
    # For analysis of spatial autocorrelation
    residuals = []

    for i in range(len(controls) + 1):
        # Subset of control variables for this iteration
        subset_controls = controls[:i]

        # Build the formula, handling the case with no controls
        controls_formula_part = " + ".join(subset_controls)
        if controls_formula_part:
            controls_formula_part = " + " + controls_formula_part

        # To avoid SettingWithCopyWarning
        df_copy = df.copy()

        # First Stage Regression
        first_stage_formula = f"{first_stage_dependent} ~ {IV}{controls_formula_part}"
        first_stage_model = smf.ols(first_stage_formula, data=df_copy)
        first_stage_fit = first_stage_model.fit()

        # Add the predicted values as a new column in the copy
        df_copy["predicted"] = first_stage_fit.predict(df_copy)

        # Second Stage Regression
        second_stage_formula = f"{second_stage_independent} ~ predicted{controls_formula_part}"
        second_stage_model = smf.ols(second_stage_formula, data=df_copy)
        second_stage_fit = second_stage_model.fit()

        def format_coef(coef, std_err, p_value):
            stars = ""
            if p_value < 0.01:
                stars = "***"
            elif p_value < 0.05:
                stars = "**"
            elif p_value < 0.1:
                stars = "*"
    
            # Check if coef or std_err is smaller than 0.000 and format accordingly
            coef_str = f"{coef:.10e}" if abs(coef) < 0.000 else f"{coef:.3f}"
            std_err_str = f"{std_err:.10e}" if abs(std_err) < 0.000 else f"{std_err:.3f}"

            return f"{coef_str}{stars} ({std_err_str})"


        # Extract coefficients and significance of first stage
        first_stage_summary = first_stage_fit.summary2().tables[1]
        first_stage_coefs = first_stage_summary.apply(lambda row: format_coef(row["Coef."], row["Std.Err."], row["P>|t|"]), axis=1)

        # Extract coefficients and significance of second stage
        second_stage_summary = second_stage_fit.summary2().tables[1]
        second_stage_coefs = second_stage_summary.apply(lambda row: format_coef(row["Coef."], row["Std.Err."], row["P>|t|"]), axis=1)

        # Adding R-squared and F-statistic
        first_stage_coefs.loc["Adj. R-squared"] = f"{first_stage_fit.rsquared_adj:.3f}"
        first_stage_coefs.loc["F-statistic"] = f"{first_stage_fit.fvalue:.3f}"
        second_stage_coefs.loc["Adj. R-squared"] = f"{second_stage_fit.rsquared_adj:.3f}"
        
        # Adding Standard Errors
        second_stage_se = second_stage_fit.bse["predicted"]
        second_stage_coefs.loc["std err"] = second_stage_se
        
        # Residuals
        res = second_stage_fit.resid
        
        # Append 
        first_stage_results.append(first_stage_coefs)
        second_stage_results.append(second_stage_coefs)
        residuals.append(res)

    # Convert the results to DataFrame
    first_stage_df = pd.DataFrame(first_stage_results).T
    second_stage_df = pd.DataFrame(second_stage_results).T
    
    # Df of residuals
    residuals_df = pd.DataFrame(residuals).T

    return first_stage_df, second_stage_df, residuals_df

## Use the Function

In [53]:
# DataFrame that contains all the variables for the regression analysis
dep_var = merged_df[["log_distance_carbon_c", 
                     "s_prot", 
                     "s_german", 
                     "longitud", 
                     "latitud", 
                     "soil_suitability", 
                     "mean_temp", 
                     "mean_preci", 
                     "dist_100km", 
                     "min_distance_uni",  
                     "BER_dummy", 
                     "density",
                     "steamengines_number",
                     "per_low",
                     "per_medium",
                     "per_unskilled",
                     "per_high",
                     "d_Hauptberuf",
                     "s_low_stud",
                     "s_high_stud",
                     "s_build",
                     "s_agri",
                     "s_mill",
                     "two",
                     "three",
                     "four",
                     "five",
                     "six",
                     "seven",
                     "ten",
                     "twelve",
                     "thirteen",
                     "fourteen"]]

In [54]:
# Make two lists of exogenous variables
# Exclude latitude and longitude once
control_list = ["longitud",
                "latitud",
                "s_prot", 
                "s_german", 
                "soil_suitability", 
                "mean_temp", 
                "mean_preci", 
                "dist_100km",
                "min_distance_uni",   
                "density",
                "s_mill",
                "s_build",
                "s_low_stud",
                "s_high_stud",
                "s_agri",
                "two",
                "three",
                     "four",
                     "five",
                     "six",
                     "seven",
                     "ten",
                     "twelve",
                     "thirteen",
                     "fourteen"]

control_list_2 = ["s_prot", 
                  "s_german", 
                  "soil_suitability", 
                  "mean_temp", 
                  "mean_preci", 
                  "dist_100km",
                  "min_distance_uni",   
                  "density",
                  "s_low_stud",
                  "s_high_stud",
                  "s_build",
                  "s_agri",
                  "s_mill",
                 "two",
                     "three",
                     "four",
                     "five",
                     "six",
                     "seven",
                     "ten",
                     "twelve",
                     "thirteen",
                     "fourteen"]

In [55]:
first_unskilled, second_unskilled, res_unskilled = twoSLS(dep_var, "log_distance_carbon_c", "steamengines_number", control_list, "per_unskilled")
first_low, second_low, res_low = twoSLS(dep_var, "log_distance_carbon_c", "steamengines_number", control_list, "per_low")
first_medium, second_medium, res_medium = twoSLS(dep_var, "log_distance_carbon_c", "steamengines_number", control_list, "per_medium")
first_high, second_high, res_high = twoSLS(dep_var, "log_distance_carbon_c", "steamengines_number", control_list, "per_high")

In [56]:
first_unskilled[[1,25]]

,1,25
Intercept,2.528*** (0.102),-3.913 (3.663)
log_distance_carbon_c,-0.087*** (0.023),-0.131*** (0.030)
Adj. R-squared,0.044,0.408
F-statistic,7.110,8.050
longitud,0.016** (0.008),0.052 (0.033)
latitud,NaN,0.065 (0.068)
s_prot,NaN,0.003*** (0.001)
s_german,NaN,0.000 (0.002)
soil_suitability,NaN,-0.138*** (0.035)
mean_temp,NaN,0.280*** (0.061)


In [57]:
second_unskilled[[1,25]]

,1,25
Intercept,4.601*** (0.278),2.545* (1.395)
predicted,-0.387*** (0.115),-0.217** (0.092)
Adj. R-squared,0.151,0.543
std err,0.114557,0.09165
longitud,0.019*** (0.003),0.026* (0.014)
latitud,NaN,0.045* (0.026)
s_prot,NaN,0.000 (0.001)
s_german,NaN,-0.000 (0.001)
soil_suitability,NaN,-0.029 (0.021)
mean_temp,NaN,-0.082** (0.037)


In [58]:
second_low[[1,25]]

,1,25
Intercept,1.711*** (0.345),6.712*** (1.876)
predicted,0.760*** (0.143),0.321*** (0.123)
Adj. R-squared,0.293,0.555
std err,0.142562,0.123255
longitud,-0.036*** (0.004),-0.053*** (0.019)
latitud,NaN,-0.073** (0.034)
s_prot,NaN,-0.001 (0.001)
s_german,NaN,-0.000 (0.001)
soil_suitability,NaN,0.069** (0.028)
mean_temp,NaN,0.038 (0.049)


In [59]:
second_medium[[1,25]]

,1,25
Intercept,2.891*** (0.227),3.043** (1.214)
predicted,0.154 (0.094),0.132* (0.080)
Adj. R-squared,0.018,0.399
std err,0.093561,0.079779
longitud,-0.005** (0.002),-0.008 (0.012)
latitud,NaN,-0.011 (0.022)
s_prot,NaN,-0.000 (0.001)
s_german,NaN,0.001** (0.001)
soil_suitability,NaN,-0.014 (0.018)
mean_temp,NaN,0.040 (0.032)


In [60]:
second_high[[1,25]]

,1,25
Intercept,-0.698 (0.563),1.655 (3.012)
predicted,0.256 (0.232),0.541*** (0.198)
Adj. R-squared,0.023,0.403
std err,0.232279,0.197943
longitud,-0.017*** (0.006),-0.068** (0.031)
latitud,NaN,-0.038 (0.055)
s_prot,NaN,-0.003** (0.001)
s_german,NaN,-0.000 (0.002)
soil_suitability,NaN,0.073 (0.044)
mean_temp,NaN,-0.003 (0.079)
